# Jupyter Notebooks and the SHARE API
----

Learn About
- Jupyter Notebooks and Python
- Making API Calls
- Using the SHARE Search API and related tools

![jupyter](img/jupyter.png)

## How YOU Can Use Jupyter Notebooks

- Learn Python and experiment with new code
- Send your code to others for them to use
- Nicely document your code using a combination of text and code blocks

## Many great resources on the web

### Jupyter/iPython Documentation
http://jupyter.readthedocs.io/en/latest

### Collections of Interesting Notebooks
https://github.com/ipython/ipython/wiki/A-gallery-of-interesting-IPython-Notebooks


## Installation

Get started by installing python on your system!

## Using Jupyter for Making API Calls

- You can use Jupyter to run any code in python (or 40+ other supported languages!)
- This workshop will focus on making calls to APIs on the web, and soon, making calls to the SHARE Search API

## API

- Application Programming Interface
- Can refer to any way to for a computer to interact with a source of data

- APIs can oftentimes be accessed over the web

![OpenNotify](img/opennotify.png)

In [3]:
import json
import requests

iss_url = 'http://api.open-notify.org/iss-now.json'

data = requests.get(iss_url).json()
print(json.dumps(data, indent=4))

{
    "message": "success",
    "iss_position": {
        "latitude": -25.905299012308827,
        "longitude": -111.90691219175612
    },
    "timestamp": 1468441770
}


In [4]:
LAT = 38.0293
LON = 78.4767

iss_url = 'http://api.open-notify.org/iss-pass.json?lat={}&lon={}'.format(LAT, LON)

data = requests.get(iss_url).json()
print(json.dumps(data, indent=4))

{
    "message": "success",
    "response": [
        {
            "risetime": 1468444588,
            "duration": 629
        },
        {
            "risetime": 1468450467,
            "duration": 527
        },
        {
            "risetime": 1468456353,
            "duration": 481
        },
        {
            "risetime": 1468462154,
            "duration": 582
        },
        {
            "risetime": 1468467929,
            "duration": 636
        }
    ],
    "request": {
        "datetime": 1468441770,
        "latitude": 38.0293,
        "longitude": 78.4767,
        "altitude": 100,
        "passes": 5
    }
}


## Parsing the Data

We got some datetimes back from the API -- but what do these mean?! 

- We can use python to find out!
- Lets use a new library, arrow, to parse that.
    + http://crsmithdev.com/arrow/

In [5]:
import arrow

![arrow_error](img/arrow_error.png)

## open your terminal
## pip install arrow

In [6]:
from arrow.arrow import Arrow


for item in data['response']:
    datetime = Arrow.fromtimestamp(item['risetime'])
    print(
        'The ISS will be visable over Charlottesville on {} at {} for {} seconds.'.format(
            datetime.date(),
            datetime.time(),
            item['duration']
        )
    )

The ISS will be visable over Charlottesville on 2016-07-13 at 17:16:28 for 629 seconds.
The ISS will be visable over Charlottesville on 2016-07-13 at 18:54:27 for 527 seconds.
The ISS will be visable over Charlottesville on 2016-07-13 at 20:32:33 for 481 seconds.
The ISS will be visable over Charlottesville on 2016-07-13 at 22:09:14 for 582 seconds.
The ISS will be visable over Charlottesville on 2016-07-13 at 23:45:29 for 636 seconds.


![pokeapi](img/pokeapi.png)

In [10]:
pokeapi = 'http://pokeapi.co/api/v2/generation/1/'

pokedata = requests.get(pokeapi).json()

# Take that data, print out a nicely formatted version of the first 10 results
print(json.dumps(pokedata['pokemon_species'][:5], indent=4))

[
    {
        "name": "bulbasaur",
        "url": "http://pokeapi.co/api/v2/pokemon-species/1/"
    },
    {
        "name": "charmander",
        "url": "http://pokeapi.co/api/v2/pokemon-species/4/"
    },
    {
        "name": "squirtle",
        "url": "http://pokeapi.co/api/v2/pokemon-species/7/"
    },
    {
        "name": "caterpie",
        "url": "http://pokeapi.co/api/v2/pokemon-species/10/"
    },
    {
        "name": "weedle",
        "url": "http://pokeapi.co/api/v2/pokemon-species/13/"
    }
]


In [12]:
# Let's get more info about the first pokemon on the list
# By following the chain of linked data

# Narrow down the url we'd like to get
bulbasaur_url = pokedata['pokemon_species'][0]['url']

# request data from that URL
bulbasaur_data = requests.get(bulbasaur_url).json()

In [13]:
print(json.dumps(bulbasaur_data, indent=4))

{
    "generation": {
        "name": "generation-i",
        "url": "http://pokeapi.co/api/v2/generation/1/"
    },
    "forms_switchable": false,
    "hatch_counter": 20,
    "growth_rate": {
        "name": "medium-slow",
        "url": "http://pokeapi.co/api/v2/growth-rate/4/"
    },
    "base_happiness": 70,
    "egg_groups": [
        {
            "name": "plant",
            "url": "http://pokeapi.co/api/v2/egg-group/7/"
        },
        {
            "name": "monster",
            "url": "http://pokeapi.co/api/v2/egg-group/1/"
        }
    ],
    "name": "bulbasaur",
    "evolution_chain": {
        "url": "http://pokeapi.co/api/v2/evolution-chain/1/"
    },
    "has_gender_differences": false,
    "flavor_text_entries": [
        {
            "language": {
                "name": "ja-kanji",
                "url": "http://pokeapi.co/api/v2/language/11/"
            },
            "flavor_text": "\u65e5\u306a\u305f\u3067\u3000\u663c\u5bdd\u3092\u3000\u3059\u308b\u3000\u59f

## Some Great APIs YOU can use!

- [Twitter](https://dev.twitter.com/overview/documentation)
- [Google Maps](https://developers.google.com/maps/web/)
- [Twillio](https://www.twilio.com/api)
- [Yelp](https://www.yelp.com/developers/documentation/v2/overview)

...and so many more!

Many require some kind of authentication, so aren't as simple as the ISS, or PokeAPI.

# SHARE Search API

Also a fantastic resource

In [14]:
SHARE_SEARCH_API = 'https://staging-share.osf.io/api/search/abstractcreativework/_search'

## The SHARE Search Schema

The SHARE search API is built on a tool called elasticsearch. It lets you search a subset of SHARE's normalized metadata in a simple format.

Here are the fields available in SHARE's elasticsearch endpoint:

    - 'title'
    - 'language'
    - 'subject'
    - 'description'
    - 'date'
    - 'date_created'
    - 'date_modified
    - 'date_updated'
    - 'date_published'
    - 'tags'
    - 'links'
    - 'awards'
    - 'venues'
    - 'sources'
    - 'contributors'

You can see a formatted version of the base results from the API by visiting the [SHARE Search API URL](http://localhost:8000/api/search/abstractcreativework/_search).